In [1]:
# Tick speed threshold model
import numpy as np
import pandas as pd
from multiprocessing import current_process

import simple.histdb as db
from simple.ohlc import debounce, npJoin
from simple.backtest import npBacktestMarket, getProfit, getLong, getShort
from simple.types import TTrade
from simple.pretty import pp
from simple.geneopt import GridOpt
from simple.plotly import interactTable, interactFigure
from simple.funcs import symlog, tickSpeed, getSpread
from simple.plotly import chartFigure, chartTrades

np.set_printoptions(edgeitems=2)
%load_ext ipycache

/home/neurox/anaconda3/lib/python3.9/site-packages/IPython/config.py:12: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  warn("The `IPython.config` package has been deprecated since IPython 4.0. "
/home/neurox/anaconda3/lib/python3.9/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [2]:
startDate, endDate = '2022-01-01', '2022-07-30'
ticker = 'BTCUSDT'
tickerID = db.getTickerID(f'{ticker}@BINANCE')
tickerID

10551

In [3]:
T = db.npTradeT(tickerID, startDate, endDate)
T.dtype = TTrade
T

  0%|          | 0/187 [00:00<?, ?it/s]

rec.array([('2022-01-13T12:34:19.993999', '2022-01-13T12:34:19.990000', 43760.82, -1., nan),
           ('2022-01-13T12:34:20.222999', '2022-01-13T12:34:20.224000', 43760.82, -1., nan),
           ...,
           ('2022-07-29T23:59:59.999000', '2022-07-29T23:59:59.996000', 23773.75, -3., nan),
           ('2022-07-29T23:59:59.999000', '2022-07-29T23:59:59.996000', 23773.75, -1., nan)],
          dtype=[('DT', '<M8[us]'), ('LocalDT', '<M8[us]'), ('Price', '<f8'), ('Size', '<f8'), ('OpenInt', '<f8')])

In [4]:
C = debounce(T)[:-1]
print(len(T), 'trades converted to', len(C), 'debounced records')
C

254910055 trades converted to 72059788 debounced records


rec.array([('2022-01-13T12:34:25.790999',        0, 43760.825, 4633001,  -8., 174., -182., 29, 13, 16),
           ('2022-01-13T12:34:29.274000',        1, 43759.465, 3483001, 320., 371.,  -51., 13,  7,  6),
           ...,
           ('2022-07-29T23:59:59.997999', 72059786, 23777.275,       0,   5.,   5.,    0.,  2,  2,  0),
           ('2022-07-29T23:59:59.997999', 72059787, 23775.705,       0,  -1.,   0.,   -1.,  1,  0,  1)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i8'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i8'), ('BuyCount', '<i8'), ('SellCount', '<i8')])

In [5]:
#%%cache mycache.pkl ts, A, vA, B, vB
ts, A, vA, B, vB = db.flatSnapS(tickerID, startDate, endDate, 1)
ts

  0%|          | 0/268054 [00:00<?, ?it/s]

array(['2022-01-13T12:34:19.277000', '2022-01-13T12:34:19.377000', ...,
       '2022-07-29T23:59:59.812999', '2022-07-29T23:59:59.912999'],
      dtype='datetime64[us]')

In [6]:
R = getSpread(ts, A[0], B[0], C).view(np.recarray)
R

rec.array([(43759.46875  , 43759.4609375, 0.07981179, 0., 0.),
           (43759.4609375, 43756.328125 , 0.11875   , 0., 0.), ...,
           (    0.       ,     0.       , 0.        , 0., 0.),
           (    0.       ,     0.       , 0.        , 0., 0.)],
          dtype=[('Ask', '<f8'), ('Bid', '<f8'), ('Mean', '<f8'), ('AskLiq', '<f8'), ('BidLiq', '<f8')])

In [7]:
np.savez_compressed(f'/tmp/{ticker}.npz', C=C, Ask=R.Ask, Bid=R.Bid)

In [8]:
# declare chart linestyles
line_styles = {
    'Price': dict(color='gray', opacity=0.25),
    'Speed': dict(color='magenta', opacity=0.5, row=2),
    'Spread': dict(color='blue', opacity=0.5, row=3),

    'Profit': dict(color='blue', width=3, opacity=0.4, secondary_y=True, connectgaps=True),
    'RawPnL': dict(color='gray', width=3, opacity=0.4, secondary_y=True, connectgaps=True),
    'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
    'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1))
}

In [9]:
def model(Threshold: int=(2, 5.5, 0.125), Period: int=(500, 15000, 500)):
    Speed = tickSpeed(C, Period, log=True)
    D = npBacktestMarket(C.DT, R.Ask, R.Bid, Speed, Threshold, hold=None)
    P = getProfit(D)
    
    return P.Profit.sum(), {
        'Count': len(P),
        'AvgProfit': P.Profit.mean() if len(P) > 0 else 0,
        'RawPnL': P.RawPnL.sum() if len(P) > 0 else 0,
        'Fee': P.Fee.sum() if len(P) > 0 else 0,
        'MidPnL': P.MidPnL.sum() if len(P) > 0 else 0,
        'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
     }

In [10]:
def modelp(Threshold: int=(2, 5.5, 0.125), Period: int=(500, 15000, 500)):
    Price = C.Price
    Speed = np.nan_to_num(tickSpeed(C, Period, log=True))
    Spread = R.Mean
    
    D = npBacktestMarket(C.DT, R.Ask, R.Bid, Speed, Threshold, hold=600_000_000)
    P = getProfit(D)
    Profit = {'x': P.Index, 'y': P.Profit.cumsum()}
    RawPnL = {'x': P.Index, 'y': P.RawPnL.cumsum()}
    Fee = {'x': P.Index, 'y': P.Fee.cumsum()}
    MidPnL = {'x': P.Index, 'y': P.MidPnL.cumsum()}
    Buy, Sell = getLong(D), getShort(D)
    
    return P.Profit.sum(), locals()

In [11]:
D = modelp(3.7, 500)
pd.DataFrame(D[1]['D'])

,X0,T0,Price0,MidPrice0,X1,T1,Price1,MidPrice1,Size
0,9289,2022-01-13 14:18:49.844,44200.000000,44187.369141,12503,2022-01-13 14:28:50.624,44189.988281,44189.994141,1.0
1,16432,2022-01-13 14:37:31.663,44346.738281,44346.734375,16433,2022-01-13 14:37:31.668,44346.730469,44346.734375,1.0
2,16434,2022-01-13 14:37:31.675,44346.730469,44346.734375,16741,2022-01-13 14:37:36.030,44390.898438,44387.353516,-1.0
3,16741,2022-01-13 14:37:36.030,44390.898438,44387.353516,16966,2022-01-13 14:37:41.125,44363.839844,44365.199219,1.0
4,16966,2022-01-13 14:37:41.125,44363.839844,44365.199219,20641,2022-01-13 14:47:41.855,44026.488281,44026.484375,-1.0
...,...,...,...,...,...,...,...,...,...
5319,71977176,2022-07-29 23:01:44.032,23928.800781,23927.410156,71977179,2022-07-29 23:01:44.051,23926.019531,23927.410156,1.0
5320,71977221,2022-07-29 23:01:44.176,23926.029297,23926.044922,71977561,2022-07-29 23:01:45.630,23939.599609,23938.375000,-1.0
5321,71977561,2022-07-29 23:01:45.630,23939.599609,23938.375000,71991508,2022-07-29 23:11:45.729,24030.560547,24030.910156,1.0
5322,72020019,2022-07-29 23:35:11.031,24017.369141,24020.219727,72032787,2022-07-29 23:45:11.137,23953.769531,23952.104492,-1.0


In [12]:
interactFigure(modelp, height=600, rows=3, **line_styles)

In [ ]:
G = GridOpt(model)
G.fullSearch()
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Fitness')
X.Threshold = X.Threshold.apply(lambda f: f'{f:1.2f}')  # explicit float format 
X

  0%|          | 0/870 [00:00<?, ?it/s]

In [ ]:
# Grid map
pp(X.pivot(index='Period', columns='Threshold', values='MidPnL').astype(int))

In [ ]:
# Grid result browser
interactTable(modelp, X, height=600, rows=3, **line_styles)